In [1]:
import os
import subprocess
import codecs

# Define the directory to search for XML files
train_dir = r'C:/Users/User/PycharmProjects/HebHTR/training_data_NEW'

# # Function to convert XML files to UTF-8
# def ensure_utf8_encoding(file_path):
#     try:
#         # Try reading the file with utf-8 encoding
#         with codecs.open(file_path, 'r', 'utf-8') as file:
#             content = file.read()
#         # Write back the content to ensure it's properly encoded in utf-8
#         with codecs.open(file_path, 'w', 'utf-8') as file:
#             file.write(content)
#     except UnicodeDecodeError:
#         # If there's an encoding error, read with a different encoding and then write in utf-8
#         with codecs.open(file_path, 'r', 'iso-8859-1') as file:
#             content = file.read()
#         with codecs.open(file_path, 'w', 'utf-8') as file:
#             file.write(content)

# Check if the directory is accessible
if not os.path.isdir(train_dir):
    print(f"Directory {train_dir} does not exist or cannot be accessed.")
    exit(1)

# Generate a list of XML files and ensure they are UTF-8 encoded
output_file = 'HebHTR_GT_NEW.txt'#'padded_output_alto.txt'
with codecs.open(output_file, 'w', 'utf-8') as f:
    for root, dirs, files in os.walk(train_dir):
        for file in files:
            if file.endswith('.xml'):
                file_path = os.path.join(root, file)
                #ensure_utf8_encoding(file_path)
                f.write(file_path + '\n')

# Check if the output.txt was created successfully
if not os.path.isfile(output_file):
    print("Failed to create output.txt. Check your permissions.")
    exit(1)



In [3]:
import os
import re
from pathlib import Path
import xml.etree.ElementTree as ET
import shutil

# Directory where the XML and image files are stored
data_dir = Path("training_data")
data_dir.mkdir(exist_ok=True)

# Load the original manifest and filter problematic filenames
manifest_path = "C:/Users/User/PycharmProjects/HebHTR/HebHTR_manifest.txt"
with open(manifest_path, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

# Characters not allowed in filenames on many systems
invalid_chars = r'[^A-Za-z0-9_.\-]'

# Track renamed files for new manifest
new_manifest_lines = []

for line in lines:
    orig_path = Path(line)
    if not orig_path.exists():
        continue

    # Clean the filename
    clean_name = re.sub(invalid_chars, '_', orig_path.name)
    new_path = data_dir / clean_name

    # Copy and clean the XML file
    if orig_path.suffix.lower() == ".xml":
        try:
            tree = ET.parse(orig_path)
            root = tree.getroot()

            # Update image filename in ALTO XML (search for fileName attribute)
            for elem in root.iter():
                if 'fileName' in elem.attrib:
                    orig_img_name = elem.attrib['fileName']
                    clean_img_name = re.sub(invalid_chars, '_', Path(orig_img_name).name)
                    elem.attrib['fileName'] = clean_img_name
                    break  # assume only one image reference

            tree.write(new_path, encoding="utf-8", xml_declaration=True)
            new_manifest_lines.append(str(new_path))

            # Also copy and rename the corresponding image if it exists
            orig_img_path = orig_path.with_suffix(".jpg")
            if not orig_img_path.exists():
                orig_img_path = orig_path.with_suffix(".png")
            if orig_img_path.exists():
                clean_img_name = re.sub(invalid_chars, '_', orig_img_path.name)
                shutil.copy2(orig_img_path, data_dir / clean_img_name)

        except ET.ParseError:
            continue  # Skip broken XMLs

# Save the new manifest
new_manifest_path = "C:/Users/User/PycharmProjects/HebHTR/HebHTR_manifest_cleaned.txt"
with open(new_manifest_path, "w", encoding="utf-8") as f:
    f.write("\n".join(new_manifest_lines))

new_manifest_path


'C:/Users/User/PycharmProjects/HebHTR/HebHTR_manifest_cleaned.txt'

In [3]:

# Compile the XML files listed in output.txt using ketos

# try:
#     result = subprocess.run(
# ['ketos compile -F C:/Users/User/PycharmProjects/PersianChildhood/output.txt -f page'], # this is the cmd command in cuda_env
# check=True)
#     if result.returncode == 0:
#         print("Process completed successfully.")
# except subprocess.CalledProcessError as e:
#     print(f"ketos compile command failed with error: {e}")
#     exit(1)


In [ ]:
#here is the cmd command: ketos compile -F C:/Users/User/PycharmProjects/PersianChildhood/output.txt --random-split 0.8 0.1 0.1 -f page
#for training, try: ketos train -f binary -d cuda:0 -r 0.0001 --resize both -i persian_best.mlmodel -o second_model dataset.arrow